# WeatherPy
----

### Analysis
* As expected, the weather becomes significantly warmer as one approaches the equator (0 Deg. Latitude). More interestingly, however, is the fact that the southern hemisphere tends to be warmer this time of year than the northern hemisphere. This may be due to the tilt of the earth.
* There is no strong relationship between latitude and cloudiness. However, it is interesting to see that a strong band of cities sits at 0, 80, and 100% cloudiness.
* There is no strong relationship between latitude and wind speed. However, in northern hemispheres there is a flurry of cities with over 20 mph of wind.

---

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import json
import sys

# Import API key
from api_keys import api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "C:/Users/kapali_s/Documents/SMU/Homeworks/Assignment_6/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [2]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)
cities

#convert the cities list to dataframe
cities = pd.DataFrame(cities, columns=["City"])

#Add columns needed
cities["Cloudiness"] = ""
cities["Country"] = ""
cities["Date"] = ""
cities["Humidity"] = ""
cities["Lat"] = ""
cities["Lng"] = ""
cities["Max Temp"] = ""
cities["Wind Speed"] = ""

cities

#write the cities to csv file
cities.to_csv("C:/Users/kapali_s/Documents/SMU/Homeworks/Assignment_6/cities.csv")

# grab a sample for development
cities_20 = cities[:20]


In [3]:
# cities.head()
cities_20.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,muscle shoals,,,,,,,,
1,saint-louis,,,,,,,,
2,tevaitoa,,,,,,,,
3,cidreira,,,,,,,,
4,olinda,,,,,,,,


### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [4]:
url = "http://api.openweathermap.org/data/2.5/weather?"
city="hilo"
query_url = f"{url}appid={api_key}&q={city}"
response = requests.get(query_url)
weather_json = response.json()
weather_json

{'coord': {'lon': -155.08, 'lat': 19.71},
 'weather': [{'id': 804,
   'main': 'Clouds',
   'description': 'overcast clouds',
   'icon': '04d'}],
 'base': 'stations',
 'main': {'temp': 296.02,
  'pressure': 1017,
  'humidity': 60,
  'temp_min': 294.82,
  'temp_max': 297.15},
 'visibility': 16093,
 'wind': {'speed': 4.1, 'deg': 70},
 'clouds': {'all': 90},
 'dt': 1553124509,
 'sys': {'type': 1,
  'id': 7881,
  'message': 0.0039,
  'country': 'US',
  'sunrise': 1553099063,
  'sunset': 1553142669},
 'id': 5855927,
 'name': 'Hilo',
 'cod': 200}

In [ ]:
#Test for London
London_cloudiness = weather_json["clouds"]["all"]
London_cloudiness
London_country = weather_json["sys"]["country"]
London_country
London_dt = weather_json["dt"]
London_dt
London_Humidity = weather_json["main"]["humidity"]
London_Humidity
London_Lat = weather_json["coord"]["lat"]
London_Lat
London_lon = weather_json["coord"]["lon"]
London_lon
London_temp_max = weather_json["main"]["temp_max"]
London_temp_max
London_wind_speed = weather_json["wind"]["speed"]
London_wind_speed

print(query_url)


In [5]:
#create a params dict that will be updated with new city each iteration
params = {"key":api_key}

for index, row in cities_20.iterrows():
    base_url = "http://api.openweathermap.org/data/2.5/weather?"
    
    city = row["City"]
    
    #make request
    query_url = f"{base_url}appid={api_key}&q={city}"
    response = requests.get(query_url)
    weather_json = response.json()
    weather_json
    
    print(f"Gathering results for: {city}")
    
    #skip missing values
    try:
        
        cities.loc[index, "Cloudiness"] = weather_json["clouds"]["all"]
        cities.loc[index, "Country"] = weather_json["sys"]["country"]
        cities.loc[index, "Date"] = weather_json["dt"]
        cities.loc[index, "Humidity"] = weather_json["main"]["humidity"]
        cities.loc[index, "Lat"] = weather_json["coord"]["lat"]
        cities.loc[index, "Lng"] = weather_json["coord"]["lon"]
        cities.loc[index, "Max Temp"]= weather_json["main"]["temp_max"]
        cities.loc[index, "Wind Speed"] = weather_json["wind"]["speed"]
        
        print(f"Success! Results for {city} added to dataframe!!\n")
             
    except Exception:
        print(f"Missing result ...skipping {sys.exc_info()} \n")
    
    cities.to_csv("C:/Users/kapali_s/Documents/SMU/Homeworks/Assignment_6/cities.csv")


Gathering results for: muscle shoals
Success! Results for muscle shoals added to dataframe!!

Gathering results for: saint-louis
Success! Results for saint-louis added to dataframe!!

Gathering results for: tevaitoa
Success! Results for tevaitoa added to dataframe!!

Gathering results for: cidreira
Success! Results for cidreira added to dataframe!!

Gathering results for: olinda
Success! Results for olinda added to dataframe!!

Gathering results for: busselton
Success! Results for busselton added to dataframe!!

Gathering results for: micheweni
Success! Results for micheweni added to dataframe!!

Gathering results for: vaini
Success! Results for vaini added to dataframe!!

Gathering results for: saint-philippe
Success! Results for saint-philippe added to dataframe!!

Gathering results for: batagay-alyta
Success! Results for batagay-alyta added to dataframe!!

Gathering results for: ngunguru
Success! Results for ngunguru added to dataframe!!

Gathering results for: punta arenas
Success!

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [6]:
cities

#print check
# cities.to_csv("C:/Users/kapali_s/Documents/SMU/Homeworks/Assignment_6/cities.csv")

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,muscle shoals,1,US,1553124367,34,34.75,-87.67,291.48,3.1
1,saint-louis,0,CH,1553124459,74,47.59,7.56,278.15,1.5
2,tevaitoa,0,PF,1553124521,100,-16.78,-151.5,302.241,1.42
3,cidreira,88,BR,1553124521,95,-30.17,-50.22,291.691,6.72
4,olinda,75,EC,1553122800,66,-2.03,-79.75,304.15,5.1
5,busselton,64,AU,1553124390,55,-33.64,115.35,293.71,7.67
6,micheweni,20,TZ,1553124522,100,-4.97,39.83,300.491,3.87
7,vaini,0,IN,1553124522,87,15.34,74.49,288.841,0.42
8,saint-philippe,75,CA,1553124290,52,45.36,-73.48,279.15,2.1
9,batagay-alyta,0,RU,1553124523,56,67.8,130.41,243.691,1.87


In [11]:
#Clean dataframe for graphs
# cities_pd = pd.read_csv("C:/Users/kapali_s/Documents/SMU/Homeworks/Assignment_6/cities.csv")
cities.replace("", np.nan, inplace=True)
cities_cleaned = cities.dropna(how="any")
cities_cleaned


,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,muscle shoals,1.0,US,1.553124e+09,34.0,34.75,-87.67,291.480,3.10
1,saint-louis,0.0,CH,1.553124e+09,74.0,47.59,7.56,278.150,1.50
2,tevaitoa,0.0,PF,1.553125e+09,100.0,-16.78,-151.50,302.241,1.42
3,cidreira,88.0,BR,1.553125e+09,95.0,-30.17,-50.22,291.691,6.72
4,olinda,75.0,EC,1.553123e+09,66.0,-2.03,-79.75,304.150,5.10
5,busselton,64.0,AU,1.553124e+09,55.0,-33.64,115.35,293.710,7.67
6,micheweni,20.0,TZ,1.553125e+09,100.0,-4.97,39.83,300.491,3.87
7,vaini,0.0,IN,1.553125e+09,87.0,15.34,74.49,288.841,0.42
8,saint-philippe,75.0,CA,1.553124e+09,52.0,45.36,-73.48,279.150,2.10
9,batagay-alyta,0.0,RU,1.553125e+09,56.0,67.80,130.41,243.691,1.87


### Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

#### Latitude vs. Temperature Plot

In [ ]:
x_axis = cities_pd["Lat"]
y_axis = cities_pd["Max Temp"]
plt.scatter(x_axis,y_axis)

#### Latitude vs. Humidity Plot

In [ ]:
x_axis = cities_pd["Lat"]
y_axis = cities_pd["Humidity"]
plt.scatter(x_axis,y_axis)

#### Latitude vs. Cloudiness Plot

In [ ]:
x_axis = cities_pd["Lat"]
y_axis = cities_pd["Cloudiness"]
plt.scatter(x_axis,y_axis)

#### Latitude vs. Wind Speed Plot

In [ ]:
x_axis = cities_pd["Lat"]
y_axis = cities_pd["Wind Speed"]
plt.scatter(x_axis,y_axis)